In [1]:
import time
import os
import math
import numpy as np
import cv2
import pandas as pd
import pyautogui as pa
import shutil
import imageio
from PIL import Image, ImageDraw

from lib.auto_GUI.auto_GUI_base import AutoGUIBase
from lib.auto_GUI.auto_DAT import AutoDAT
from lib.auto_GUI.auto_PhotoZ import AutoPhotoZ
from lib.analysis.laminar_dist import *
from lib.file.TIF import *
from lib.analysis.align import ImageAlign
from lib.utilities import *

In [2]:
""" 
    Given an SNR cutoff, find the area (# pixels) that is above cutoff.
        Input: ZDA files (need to export every SNR frame from start frame to end frame)
        Output:
            A(t) in a data file and a plot
            Movies of the change of this area in frames, two-color frames
            
    Update: instead of counting # pixels with SNR > 5, can count total SNR.
"""
data_dir = "C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/"
start_frame = 94
movie_length = 27
snr_cutoff = 0.85  # if None, instead of counting # pixels with SNR > 5, count total SNR.
                   # if < 1, percentile

disable_photoZ_interact = False
initialize_photoZ = False
clean_scratch_files = False  # clean up SNR maps
replace_existing_frame_dats = False

In [3]:
# set up PhotoZ (open it manually)
aPhz = AutoPhotoZ(data_dir=data_dir)
if initialize_photoZ and not disable_photoZ_interact:
    aPhz.prepare_photoZ()
if not disable_photoZ_interact:
    aPhz.click_normalize_2_array_max()
pa.alert("Please disable trace overlay and maximize PhotoZ before continuing.")

'OK'

In [ ]:
snr_dict = {}

for subdir, dirs, files in os.walk(data_dir):
    if 'notUsable' in subdir:
        continue
    if 'mm_hidden' in subdir:
        continue
    if 'converted_zda' in subdir:
        continue

    for zda_file in files:
        if not zda_file.endswith('.zda'):
            continue

        rec_id = zda_file.split('.')[0]
        print(rec_id)
        # movie dir
        movie_dir = subdir + "/responseArea" + rec_id + "/"

        slic, loc, rec = [int(x) for x in rec_id.split("_")]
        print(slic, loc, rec)
        try:
            os.makedirs(movie_dir)
        except Exception as e:
            pass

        if not disable_photoZ_interact:
            aPhz = AutoPhotoZ(data_dir=subdir)


            need_to_open_zda = False
            for frame in range(start_frame, start_frame + movie_length):
                f = aPhz.pad_zeros(slic) + "_" + aPhz.pad_zeros(loc) + "_" + aPhz.pad_zeros(rec) + ".dat"
                f = subdir + "/SNR_" + rec_id + "_frame" + str(frame)+"_" + f
                if not os.path.exists(f):
                    need_to_open_zda = True

            need_to_open_zda = (need_to_open_zda or replace_existing_frame_dats)
            if need_to_open_zda:
                # open the PhotoZ file
                aPhz.select_PhotoZ()

                print("\n\nOpening", zda_file)
                aPhz.open_zda_file(subdir + "/" + zda_file)

                # set measure window width to 1
                aPhz.set_measure_window(None, 1)
            for frame in range(start_frame, start_frame + movie_length):
                f = aPhz.pad_zeros(slic) + "_" + aPhz.pad_zeros(loc) + "_" + aPhz.pad_zeros(rec) + ".dat"
                f = subdir + "/SNR_" + rec_id + "_frame" + str(frame)+"_" + f
                if not os.path.exists(f):
                    aPhz.set_measure_window(frame, None)
                    aPhz.select_array_tab()
                snr_filename = AutoDAT(datadir=subdir,
                        file_prefix="SNR_" + rec_id + "_frame" + str(frame)+"_").save_background(slic, 
                                                                                                 loc, 
                                                                                                 rec,
                                                                                                 overwrite_existing=replace_existing_frame_dats)
                if subdir not in snr_dict:
                    snr_dict[subdir] = {}
                if rec_id not in snr_dict[subdir]:
                    snr_dict[subdir][rec_id] = {}
                if frame not in snr_dict[subdir][rec_id]:
                    snr_dict[subdir][rec_id][frame] = { 'filename': snr_filename }

02_01_01
2 1 1
02_01_02
2 1 2
02_01_04
2 1 4
04_01_03
4 1 3
04_01_07
4 1 7


Opening 04_01_07.zda


In [ ]:
for subdir, dirs, files in os.walk(data_dir):
    if 'notUsable' in subdir:
        continue
    if 'mm_hidden' in subdir:
        continue
    if 'converted_zda' in subdir:
        continue
    
    for zda_file in files:
        if not zda_file.endswith('.zda'):
            continue

        rec_id = zda_file.split('.')[0]
        print(rec_id)
        # movie dir
        movie_dir = subdir + "/responseArea" + rec_id + "/"
        
        slic, loc, rec = [int(x) for x in rec_id.split("_")]
        print(slic, loc, rec)
        
        snr_frames = []
        areas_over_time = []
        images = []
        
        all_min, all_max = [], []
        
        snr_frames_buffer = []
        for frame in range(start_frame, start_frame + movie_length):
            snr_filename = snr_dict[subdir][rec_id][frame]['filename']

            snr_frame = np.loadtxt(snr_filename,
                         delimiter='\t',
                         usecols=(1,))
            width = int(np.sqrt(snr_frame.shape[0]))
            snr_frame = snr_frame.reshape((width, width))
            all_min.append(np.min(snr_frame))
            all_max.append(np.max(snr_frame))
            snr_frames_buffer.append(snr_frame)
        all_min = min(all_min)
        all_max = min(all_max)
        
        if snr_cutoff is not None:
            if snr_cutoff > 1:
                sc = snr_cutoff

            else:
                # percentile
                sc = np.percentile(np.concatenate(snr_frames_buffer), int(snr_cutoff * 100))
        
        for i in range(len(snr_frames_buffer)):
            frame = i + start_frame
            snr_frame = snr_frames_buffer[i]
            width = snr_frame.shape[0]
            if snr_cutoff is not None:
                snr_frame = np.ma.masked_less(snr_frame, sc)
                    
            area = np.sum(snr_frame)
            snr_frames.append(snr_frame)
            areas_over_time.append(area)
            # print(snr_frame, area)
            plt.clf()
            plt.imshow(np.zeros(snr_frame.shape))
            im = plt.imshow(snr_frame, 
                            vmin=all_min,
                            vmax=all_max)
            cbar = plt.colorbar(im, label="SNR ", ticks=[all_min, all_max])
            cbar.ax.set_yticklabels([str(all_min), str(all_max)])
            fn_image = movie_dir + str(frame) + ".png"
            plt.savefig(fn_image)
            images.append(imageio.imread(fn_image))
            
        # plot A(t)
        print(subdir)
        plt.clf()
        plt.plot([i * 0.5 for i in range(start_frame, start_frame + movie_length)], 
                 areas_over_time)
        plt.xlabel("Time (ms)")
        plt.ylabel("Total signal (pixels)")
        plt.title("Responding Area over Time")
        plt.savefig(movie_dir + "area.png")
        plt.show()
        
        # Make boolean signal movie (animated gif)
        created_movie = False
        try:
            imageio.mimsave(movie_dir + 'area_movie.gif', images)
            print("CREATED MOVIE:", rec_id + 'area_movie.gif')
            created_movie = True
            if clean_scratch_files:
                for img_file in img_filenames:
                    os.remove(img_file)
        except Exception as e:
            if not created_movie:
                print("Not creating movie for " + rec_id)
            print(e)
        